## Example: Reading tiled raster data with different zoom levels

This example will show how one can export raster dataset to individual tiles at differnt zoom levels and read the data via the via the [DataCatalog.get_rasterdataset](https://deltares.github.io/hydromt/latest/_generated/hydromt.data_catalog.DataCatalog.get_rasterdataset.html) method. 

In [ ]:
# Imports
from hydromt import raster, DataCatalog
from hydromt.log import setuplog
from os.path import join

logger = setuplog('tiling', log_level=20)

# get some elevation data from the data catalog
data_lib = "artifact_data"
data_cat = DataCatalog(data_lib, logger=logger)
source = "merit_hydro"
da0 = data_cat.get_rasterdataset(source, variables=["elevtn"])
da0.raster.shape

`da0` is gridded data as a xarray.DataArray object. 
With HydroMT a xarray.DataArray has some extra functionality via `.raster`
This extra functionality does include the ability to write a raster to a tile database (tiling).


### tiling raster with XYZ stucture

First let's have a look at the XYZ structure.
A xarray.DataArray is simple written to a tile database in XYZ structure via .raster.to_xyz_tiles



In [ ]:
# Write the database in XYZ stucture
name = f"{source}_xyz"
root = join("tmpdir", name)
zoom_levels = [0, 1, 2, 3, 5]  # note that you can skip a zoom level
da0.raster.to_xyz_tiles(
    root=root,
    tile_size=256,
    zoom_levels=zoom_levels,
    driver="GTiff",  # try also 'netcdf4'
    compress="deflate",
)

The tiles in the 'merit_hydro_xyz' folder now contains the zoom_levels as defined above. 

zoomlevel 0 is at the scale of the xarray.DataArray (one on one). zoomlevel 1 is downscaled by a factor 2 compared to zoomlevel 0. zoomlevel 3 is downscaled by a factor of 8 compared to zoomlevel 0, etc.

A mosaic is created per zoomlevel of these tiles in a .vrt file.

At last, a .yml file is produced which can be read by the [DataCatalog](https://deltares.github.io/hydromt/latest/_generated/hydromt.data_catalog.DataCatalog.html) of HydroMT.

### reading tiled raster data with zoom levels

With DataCatalog.[get_rasterdataset](https://deltares.github.io/hydromt/latest/_generated/hydromt.data_catalog.DataCatalog.get_rasterdataset.html) a raster (.vrt) can be retreived. In case of a tile database it can be done for a certain zoomlevel. E.g.

In [ ]:
# Imports
from hydromt import DataCatalog

# Load the yml into a DataCatalog
data_catalog = DataCatalog(join(root, f"{name}.yml"), logger=logger)

# View the structure of the DataCatalog
data_catalog[name]

In [ ]:
# without zoom_level the highest res data is fetched
da0 = data_catalog.get_rasterdataset(name)
da0.raster.shape

In [ ]:
# Request a raster from the Datacatolog based on zoom resolution & unit
da = data_catalog.get_rasterdataset(name, zoom_level=(1/600, 'degree'))

da = data_catalog.get_rasterdataset(name, zoom_level=(1e3, 'meter'))


In [ ]:
# We can also directly request a specific zoom level
da = data_catalog.get_rasterdataset(name, zoom_level=zoom_levels[-1])
print(da.raster.shape)

In [ ]:
# View the data
import matplotlib.pyplot as plt

fig, (ax, ax1) = plt.subplots(1, 2, figsize=(8, 4))
da0.raster.mask_nodata().plot.imshow(ax=ax, vmin=0, vmax=2500, add_colorbar=False)
ax.set_title(f"zoomlevel {zoom_levels[0]}")
da.raster.mask_nodata().plot.imshow(ax=ax1, vmin=0, vmax=2500, add_colorbar=False)
ax1.set_title(f"zoomlevel {zoom_levels[-1]}")

### Caching tiled raster datasets

Tiles of tiled rasterdatasets which are described by a .vrt file can be cached locally (starting from v0.7.0). 
The requested data tiles will by default be stored to ~/.hydromt_data.

In [ ]:
# set caching to True
# NOTE this can also be done at initialization of the DataCatalog
data_catalog.cache = True

# request some tiles based on bbox -> note the log messages
da0 = data_catalog.get_rasterdataset(name, bbox=[11.6, 45.3, 12.0, 46.0])

In [ ]:
# if we run the same request again we will use the cached files (and download none) 
da0 = data_catalog.get_rasterdataset(name, bbox=[11.6, 45.3, 12.0, 46.0])